In [3]:
import numpy as np
def featureSignSearch(matA, vecY, gamma):
    # FINDING ATA(A transpose A)
    AtransA = np.dot(matA.T, matA)
    # TO find (ATA)-1 * (ATy-gamma*thetahat/2)
    getTheValue = np.dot(matA.T, vecY)

    x = np.zeros(AtransA.shape[0])
    # Initializing the initial theta vector
    theta = np.zeros(AtransA.shape[0], dtype=np.int8)
    active_set = set()

    #Finding the derivative
    derivativeOfTheCorr = - 2 * getTheValue

    sds = np.dot(vecY.T, vecY)
    
    #INitializing the global optimum value
    globalOptimum = np.inf
    nglobalOptimum = 0

    #From zero coefficients of x, select i = arg maxi for the derivative
    #Check the optimality conditions
    
    while globalOptimum > gamma or not np.allclose(nglobalOptimum, 0):
        if np.allclose(nglobalOptimum, 0):
            id = np.argmax(np.abs(derivativeOfTheCorr) * (theta == 0))
            if derivativeOfTheCorr[id] > gamma:
                theta[id] = -1.
                x[id] = 0.
                active_set.add(id)
            elif derivativeOfTheCorr[id] < -gamma:
                theta[id] = 1.
                x[id] = 0.
                active_set.add(id)
            if len(active_set) == 0:
                break

        indices = np.array(sorted(active_set))
        AtransAOriginal = AtransA[np.ix_(indices, indices)]
        getTheValueOriginal = getTheValue[indices]
        signOriginal = theta[indices]
        onTheRight = getTheValueOriginal - gamma * signOriginal / 2
        
        xbar = np.linalg.solve(np.atleast_2d(AtransAOriginal), onTheRight)
        new_theta = np.sign(xbar)
        oldSolution = x[indices]
        changesOfSign = np.where(abs(new_theta - signOriginal) > 1)[0]
        
        #Feature-sign step:
        
        if len(changesOfSign) > 0:
            trueValueOfObject = np.inf
            trueValueOfCurrent = None
            trueValueOfCurrent = xbar
            trueValueOfObject = (sds + (np.dot(xbar,np.dot(AtransAOriginal, xbar))- 2 * np.dot(xbar, getTheValueOriginal))+ gamma * abs(xbar).sum())
            #Checking for any changes in the sign of the objective value
            for idx in changesOfSign:
                a = xbar[idx]
                b = oldSolution[idx]
                value = b / (b - a)
                curr = oldSolution - value * (oldSolution - xbar)
                toBeMinimized = sds + (np.dot(curr, np.dot(AtransAOriginal, curr))- 2 * np.dot(curr, getTheValueOriginal)+ gamma * abs(curr).sum())
                
                if toBeMinimized < trueValueOfObject:
                    trueValueOfObject = toBeMinimized
                    trueValueOfCurrent = curr
        else:
            trueValueOfCurrent = xbar
        x[indices] = trueValueOfCurrent

        zeros = indices[np.abs(x[indices]) < 1e-8]
        x[zeros] = 0.
        theta[indices] = np.int8(np.sign(x[indices]))
        active_set.difference_update(zeros)
        derivativeOfTheCorr = - 2 * getTheValue + 2 * np.dot(AtransA, x)
        globalOptimum = np.max(abs(derivativeOfTheCorr[theta == 0]))
        nglobalOptimum = np.max(abs(derivativeOfTheCorr[theta != 0] + gamma * theta[theta != 0]))
    return x

In [6]:
# GENERATING THE RANDOM MATRIX SIZE NXN
N=25
mylist1 = []
for i in range(N):
    mylist1.append(np.random.rand(N))
mat = np.array(mylist1)
#GENERATING THE RANDOM VECTOR NX1
mylist2 = np.empty([0,1])
for i in range(N):
    mylist2=np.append(mylist2, np.random.rand(1))
y = np.array(mylist2)

#INITIAL GAMMA VALUE
gamma=0.5

x_final=featureSignSearch(mat,y,gamma)

print("Final output solution is :\n",x_final)

Final output solution is :
 [ 0.          0.18392789  0.02817168  0.19529864  0.03704045  0.09082723
  0.20974196  0.          0.          0.         -0.0385899  -0.03244067
  0.          0.10764655  0.          0.2608894   0.         -0.04967106
  0.          0.         -0.0241941   0.         -0.04545131  0.13479428
  0.        ]
